In [1]:
import re
from urllib.parse import urlparse

import pandas as pd
import tensorflow

In [2]:
def fd_length(url):
    urlpath = urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0


def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits


def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters


def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')


def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return -1
    else:
        # print 'No matching pattern found'
        return 1


def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return -1
    else:
        return 1


def preprocess_data(filepath):
    urldata = pd.read_csv(filepath)
    urldata.head()
    urldata.head()
    urldata.shape
    urldata.info()
    urldata.isnull().sum()
    urldata['url_length'] = urldata['url'].apply(lambda i: len(str(i)))
    urldata['hostname_length'] = urldata['url'].apply(lambda i: len(urlparse(i).path))
    urldata['path_length'] = urldata['url'].apply(lambda i: len(urlparse(i).path))
    urldata['fd_length'] = urldata['url'].apply(lambda i: fd_length(i))
    urldata.head()
    urldata['count-'] = urldata['url'].apply(lambda i: i.count('-'))
    urldata['count@'] = urldata['url'].apply(lambda i: i.count('@'))
    urldata['count?'] = urldata['url'].apply(lambda i: i.count('?'))
    urldata['count%'] = urldata['url'].apply(lambda i: i.count('%'))
    urldata['count.'] = urldata['url'].apply(lambda i: i.count('.'))
    urldata['count='] = urldata['url'].apply(lambda i: i.count('='))
    urldata['count-http'] = urldata['url'].apply(lambda i: i.count('http'))
    urldata['count-https'] = urldata['url'].apply(lambda i: i.count('https'))
    urldata['count-www'] = urldata['url'].apply(lambda i: i.count('www'))
    urldata['count-digits'] = urldata['url'].apply(lambda i: digit_count(i))
    urldata['count-letters'] = urldata['url'].apply(lambda i: letter_count(i))
    urldata['count_dir'] = urldata['url'].apply(lambda i: no_of_dir(i))
    urldata['use_of_ip'] = urldata['url'].apply(lambda i: having_ip_address(i))
    urldata['short_url'] = urldata['url'].apply(lambda i: shortening_service(i))
    x = urldata[['hostname_length',
                 'path_length', 'fd_length', 'count-', 'count@', 'count?',
                 'count%', 'count.', 'count=', 'count-http', 'count-https', 'count-www', 'count-digits',
                 'count-letters', 'count_dir', 'use_of_ip']]

    y = urldata['result']
    x_sample = pd.DataFrame(x)
    y_sample = pd.DataFrame(y)
    return x_sample, y_sample

In [3]:
model = tensorflow.keras.models.load_model("IA_models/Ashish_Arya.h5")
x_sample, y_sample = preprocess_data("data/full_history.csv")

history_loss, history_accuracy = model.evaluate(x_sample, y_sample, batch_size=256)

x_sample_bis, y_sample_bis = preprocess_data("data/urldata.csv")

kaggle_loss, kaggle_accuracy = model.evaluate(x_sample_bis, y_sample_bis, batch_size=256)

print("Ashish_Arya")
print("Accuracy for history: ", history_accuracy)
print("Accuracy for kaggle: ", kaggle_accuracy)
#history=model.fit(x_train, y_train, batch_size=256, epochs=5, validation_data=(x_test, y_test), callbacks=[checkpointer])
#str(model.(np.array([x,])))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12862 entries, 0 to 12861
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     12862 non-null  object
 1   result  12862 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 201.1+ KB
51/51 [==============================] - 0s 669us/step - loss: 2.2750 - acc: 0.5058
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  450176 non-null  int64 
 1   url         450176 non-null  object
 2   label       450176 non-null  object
 3   result      450176 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 13.7+ MB
1759/1759 [==============================] - 1s 617us/step - loss: 0.0213 - acc: 0.9958